In [1]:
from __future__ import annotations

import collections
import hashlib
import itertools
import json
import sys
from copy import deepcopy
from pathlib import Path
from time import time
from math import comb
from functools import reduce

import bokeh
import hiplot as hip
import matplotlib
import numpy as np
import pandas as pd
import pandas_bokeh
import ray
from bson.objectid import ObjectId
from matplotlib import pyplot as plt
from omegaconf import OmegaConf
from pymongo import MongoClient
from math import comb
from functools import reduce
import seaborn as sns

import xplogger

print(xplogger.__version__)
from xplogger.experiment_manager.record.record_list import RecordList
from xplogger.experiment_manager.store.mongo import MongoStore
from xplogger.experiment_manager.viz import bokeh as bokeh_viz
from xplogger.experiment_manager.notebook import utils as notebook_utils
from xplogger.parser.experiment import (
    Experiment,
    ExperimentSequence,
    ExperimentSequenceDict,
)
from xplogger.parser.experiment import utils as experiment_utils

sys.path.append("/private/home/sodhani/projects/abstraction_by_gating/")

from math import sqrt
from time import time

from IPython.display import JSON
from notebooks import utils, ds
from notebooks import svd as svd_utils

0.10.0


In [2]:
%matplotlib inline
pd.set_option("plotting.backend", "pandas_bokeh")
pandas_bokeh.output_notebook()

%matplotlib inline
# pd.set_option("plotting.backend", "pandas_bokeh")
# sns.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
# pandas_bokeh.output_notebook()
sns.color_palette("deep")

plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

Loading BokehJS ...

In [3]:
collection_name = "abstraction_by_gating"

use_record_ids = False
ray.init(num_cpus=20, ignore_reinit_error=True)

E0605 15:07:32.666611292 2236924 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0605 15:07:32.701396015 2236924 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0605 15:07:32.725900863 2236924 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


{'node_ip_address': '100.97.66.230',
 'raylet_ip_address': '100.97.66.230',
 'redis_address': '100.97.66.230:29665',
 'object_store_address': '/tmp/ray/session_2022-06-05_15-07-31_492088_2236924/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-06-05_15-07-31_492088_2236924/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-06-05_15-07-31_492088_2236924',
 'metrics_export_port': 55127,
 'gcs_address': '100.97.66.230:39805',
 'node_id': 'f8f4df7d5f4a82adb95ed7a692ecd3749f5bf4a433d07bb1b26fe6d5'}

In [4]:
mongo_store = MongoStore(
    config={
        "host": "localhost",
        "port": 27017,
        "db": "project",
        "collection_name": collection_name,
    }
)
print(f"Num documents: {mongo_store.collection.count_documents({})}")

Num documents: 43900


In [5]:
# start_time = time()
# mongo_records = mongo_store.ray_get_records()
# print(time() - start_time)
start_time = time()
mongo_records = mongo_store.get_records(query={})
print(time() - start_time)

5.600881099700928


In [6]:
mongo_to_record = {}
issue_to_mongo_record = {}
unanalyzed_records = []
start = time()
for record in mongo_records:
    mongo_record_id = record.id
    mongo_to_record[mongo_record_id] = record
    issue_id = str(record["setup"]["git"]["issue_id"])
    if issue_id not in issue_to_mongo_record:
        issue_to_mongo_record[issue_id] = []
    issue_to_mongo_record[issue_id].append(record)
    if record["status"] != "ANALYZED":
        unanalyzed_records.append(record)

print(f"Time taken: {time() - start}")
assert len(unanalyzed_records) == len(mongo_store.get_unanalyzed_records())

Time taken: 0.20273184776306152


In [7]:
start = time()
record_list = RecordList(unanalyzed_records[:])
print(f"Total number of unanalayzed records: {len(record_list)}")
unique_issues = record_list.get_unique_issues()
print(unique_issues)
print(f"Time taken: {time() - start}")

Total number of unanalayzed records: 32475
Counter({'1': 15942, '3': 4809, '25': 2751, '4': 2304, '36': 1408, '28': 1152, '29': 1123, '22': 962, '35': 614, '3-1': 480, '121': 372, 'dummy': 220, '31': 168, '18': 158, '30': 12})
Time taken: 0.07647252082824707


In [8]:
# start = time()
record_list = []
# for issue_id in ["335", "325"]:
for issue_id in ["25"]:
    # for issue_id in issue_to_mongo_record:
    record_list += issue_to_mongo_record[issue_id]


def key_func(record):
    return record["setup"]["id"]

def filter_fn(record):
    return True
#     return record["dataloader"]["task_specific_cfgs"]["living_or_not"]["train_config"]["model"]["should_use_pretrained"]
#     return record["setup"]["script_id"] not in ["3-3", "3-4"]

print(len(record_list))
record_list = RecordList([x for x in record_list if filter_fn(x)])
# RecordList([x for x in record_list.ray_make_oc_records() if x["status"]!="ANALYZED" and "id" in x]).mark_analyzed(collection=mongo_store.collection)
slurm_id_to_record_map = record_list.map_to_slurm_id()
record_list = record_list.get_unique(key_func).ray_make_oc_records()
# print(f"Time taken: {time() - start}")
print(f"Number of selected records: {len(record_list)}")
print(
    f"Number of uniques runs: {len(set(record['setup']['id'] for record in record_list))}"
)
print(
    f"Number of unanalyzed runs: {len(set(record['setup']['id'] for record in record_list if record['status']!='ANALYZED'))}"
)
print(
    f"Set of issues: {set(record['setup']['git']['issue_id'] for record in record_list)}"
)
print(
    f"Set of script ids: {set(record['setup']['script_id'] for record in record_list)}"
)

2751
Number of selected records: 449
Number of uniques runs: 449
Number of unanalyzed runs: 449
Set of issues: {25}
Set of script ids: {25, '25-1'}


In [9]:
record_list = RecordList(list(x for x in record_list if x.setup.script_id == 25))

In [10]:
len(record_list)

240

In [11]:
load_experiment_from_dir = lambda log_dir: utils.load_experiment_from_dir(
    log_dir=log_dir,
    cache_failed_experiment=True,
    verbose=False,
    skip_cache=False,
)

viz_params = record_list.get_viz_params()

viz_params.remove("experiment")
start_time = time()
# (
#     exp_seq_dict,
#     groups,
#     hyperparams,
# ) = record_list.ray_make_experiment_sequence_dict_groups_and_hyperparams(
#     viz_params, load_experiment_from_dir
# )

(
    exp_seq_dict,
    groups,
    hyperparams,
) = record_list.make_experiment_sequence_dict_groups_and_hyperparams(
    viz_params, load_experiment_from_dir
)

print(f"Time taken: {time() - start_time}")


notebook_utils.prettyprint_dict(hyperparams)

Time taken: 7.572144269943237
setup.script_id
	{25}
model.hidden_layer_cfg.dim
	{1024, 128}
experiment.task.mode
	{'rotate_input_permute_target', 'permute_input_permute_target'}
model.num_layers
	{1}
model.weight_init.should_do
	{True}
dataloader.name
	{'mnist'}
optimizer._target_
	{'torch.optim.SGD'}
experiment.num_epochs
	{1000}
optimizer.lr
	{0.0001}
experiment.task.num_input_transformations
	{40}
model.should_use_non_linearity
	{False}
model.hidden_layer_cfg.should_share
	{True}
dataloader.train_config.dataloader.batch_size
	{8}
model.gate_cfg.mode
	{'10_plus_mod_permute', '30_plus_mod_permute', '35_plus_mod_permute', '15_plus_mod_permute', '20_plus_mod_permute', '25_plus_mod_permute', '5_plus_mod_permute'}
experiment.task.num_classes_in_selected_dataset
	{10}
model.decoder_cfg.should_share
	{False}
model.weight_init.gain
	{0.1, 1.0, 10.0, 0.01, 0.0001, 0.001}
optimizer.momentum
	{0.9}
model.pretrained_cfg.should_use
	{False}
model.weight_init.bias
	{0}
model.encoder_cfg.should_sha

In [12]:
viz_params

{'dataloader.name',
 'dataloader.train_config.dataloader.batch_size',
 'experiment.num_epochs',
 'experiment.task.mode',
 'experiment.task.num_classes_in_selected_dataset',
 'experiment.task.num_input_transformations',
 'model.decoder_cfg.should_share',
 'model.encoder_cfg.should_share',
 'model.gate_cfg.mode',
 'model.hidden_layer_cfg.dim',
 'model.hidden_layer_cfg.should_share',
 'model.num_layers',
 'model.pretrained_cfg.should_use',
 'model.should_use_non_linearity',
 'model.weight_init.bias',
 'model.weight_init.gain',
 'model.weight_init.should_do',
 'optimizer._target_',
 'optimizer.lr',
 'optimizer.momentum',
 'setup.script_id'}

In [13]:
print(list(exp_seq_dict.values())[0])
print(type(list(exp_seq_dict.values())[0]))

[<xplogger.parser.experiment.experiment.Experiment object at 0x7fbc35a615b0>, <xplogger.parser.experiment.experiment.Experiment object at 0x7fbc35943fa0>]
<class 'xplogger.parser.experiment.experiment.ExperimentSequence'>


In [14]:
print(list(exp_seq_dict.values())[0][0].config["setup"]["id"])

ab14dfb9c02ab4b3e2c3c5483ad8b0ae20fccb2644d8346b78dda6c9_issue_25_seed_1


In [15]:
exp = list(exp_seq_dict.values())[0][0]
pd.options.display.max_columns = None
pd.options.display.max_rows = None
exp.metrics["train_epoch"]

,average_loss_for_selected_paths,average_loss_for_unselected_paths,average_accuracy_for_selected_paths,average_accuracy_for_unselected_paths,gating_matrix,epoch,mode,step,time_taken,logbook_id,logbook_timestamp,logbook_type
0,2.302608,2.302608,0.112103,0.112100,0,0,train,7499,205.416527,0,"09:08:38PM PST Nov 21, 2021",metric
2,2.302505,2.302505,0.112367,0.112367,0,1,train,14999,207.248248,0,"09:12:26PM PST Nov 21, 2021",metric
4,2.302456,2.302456,0.112367,0.112367,0,2,train,22499,205.169314,0,"09:16:11PM PST Nov 21, 2021",metric
6,2.302411,2.302411,0.112367,0.112367,0,3,train,29999,211.099775,0,"09:20:03PM PST Nov 21, 2021",metric
8,2.302364,2.302364,0.112367,0.112367,0,4,train,37499,206.811648,0,"09:23:51PM PST Nov 21, 2021",metric
10,2.302314,2.302314,0.112367,0.112367,0,5,train,44999,204.385393,0,"09:27:36PM PST Nov 21, 2021",metric
12,2.302275,2.302275,0.112367,0.112367,0,6,train,52499,217.718355,0,"09:31:34PM PST Nov 21, 2021",metric
14,2.302233,2.302233,0.112367,0.112367,0,7,train,59999,214.916396,0,"09:35:31PM PST Nov 21, 2021",metric
16,2.302194,2.302194,0.112367,0.112367,0,8,train,67499,206.435869,0,"09:39:16PM PST Nov 21, 2021",metric
18,2.302151,2.302151,0.112367,0.112367,0,9,train,74999,201.687643,0,"09:42:57PM PST Nov 21, 2021",metric


In [16]:
list(exp_seq_dict.values())[0][0].config["setup"]["viz"]

{'params': ['experiment.num_epochs',
  'experiment.task.num_input_transformations',
  'experiment.task.num_classes_in_selected_dataset',
  'dataloader.train_config.dataloader.batch_size',
  'dataloader.name',
  'experiment',
  'model.num_layers',
  'model.gate_cfg.mode',
  'model.pretrained_cfg.should_use',
  'experiment.task.mode',
  'model.hidden_layer_cfg.dim',
  'model.should_use_non_linearity',
  'model.encoder_cfg.should_share',
  'model.hidden_layer_cfg.should_share',
  'model.decoder_cfg.should_share',
  'setup.script_id',
  'model.weight_init.should_do',
  'model.weight_init.gain',
  'model.weight_init.bias',
  'optimizer._target_',
  'optimizer.lr',
  'optimizer.momentum']}

In [17]:
notebook_utils.prettyprint_dict(hyperparams)

setup.script_id
	{25}
model.hidden_layer_cfg.dim
	{1024, 128}
experiment.task.mode
	{'rotate_input_permute_target', 'permute_input_permute_target'}
model.num_layers
	{1}
model.weight_init.should_do
	{True}
dataloader.name
	{'mnist'}
optimizer._target_
	{'torch.optim.SGD'}
experiment.num_epochs
	{1000}
optimizer.lr
	{0.0001}
experiment.task.num_input_transformations
	{40}
model.should_use_non_linearity
	{False}
model.hidden_layer_cfg.should_share
	{True}
dataloader.train_config.dataloader.batch_size
	{8}
model.gate_cfg.mode
	{'10_plus_mod_permute', '30_plus_mod_permute', '35_plus_mod_permute', '15_plus_mod_permute', '20_plus_mod_permute', '25_plus_mod_permute', '5_plus_mod_permute'}
experiment.task.num_classes_in_selected_dataset
	{10}
model.decoder_cfg.should_share
	{False}
model.weight_init.gain
	{0.1, 1.0, 10.0, 0.01, 0.0001, 0.001}
optimizer.momentum
	{0.9}
model.pretrained_cfg.should_use
	{False}
model.weight_init.bias
	{0}
model.encoder_cfg.should_share
	{False}


In [18]:
def map_combination_to_hyperparams(combination):
    combination_dict = {}
    for key, value in zip(hyperparams.keys(), combination):
        combination_dict[key] = value
    return combination_dict

In [19]:
def make_metadata(
    mode: str,
    max_epochs: int,
    min_epochs: int,
    metric_name: str,
):
    metadata = OmegaConf.create(
        {
            "metric_name": metric_name,
            "mode": mode,
            "x": {"name": "epoch", "min": min_epochs, "max": max_epochs},
        }
    )

    step_metadata = OmegaConf.create(
        {
            "metric_name": "epoch",
            "mode": mode,
            "x": {"name": "epoch", "min": min_epochs, "max": max_epochs},
        }
    )

    return metadata, step_metadata

In [20]:
# list(exp_seq_dict.values())[0].aggregate_metrics(
#         metric_names=[metadata.metric_name],
#         x_name=metadata.x.name,
#         x_min=metadata.x.min,
#         x_max=metadata.x.max,
#         mode=metadata.mode,
#         drop_duplicates=True,
#         verbose=True,
#     )

In [21]:
type(list(exp_seq_dict.values())[0])

xplogger.parser.experiment.experiment.ExperimentSequence

In [22]:
mode = "test_epoch"
# metric_name = "accuracy_task_two_encoder_task_one_decoder"

metrics = [
    "average_accuracy_for_selected_paths",
    "average_accuracy_for_unselected_paths",
    "average_loss_for_selected_paths",
    "average_loss_for_unselected_paths",
]

df_dict = {}

for metric_name in metrics:
    metadata, step_metadata = make_metadata(
        mode=mode,
        max_epochs=1000,
        min_epochs=1,
        metric_name=metric_name,
    )

    df_dict[metric_name] = notebook_utils.make_df(
        metadata=metadata,
        step_metadata=step_metadata,
        groups=groups,
        hyperparams=hyperparams,
        exp_seq_dict=exp_seq_dict,
    )
    
#     df_dict[key].drop(
#         [],
#         axis=1,
#         inplace=True,
#     )

In [23]:
df_dict["average_accuracy_for_unselected_paths"]

,setup.script_id,model.hidden_layer_cfg.dim,experiment.task.mode,model.num_layers,model.weight_init.should_do,dataloader.name,optimizer._target_,experiment.num_epochs,optimizer.lr,experiment.task.num_input_transformations,model.should_use_non_linearity,model.hidden_layer_cfg.should_share,dataloader.train_config.dataloader.batch_size,model.gate_cfg.mode,experiment.task.num_classes_in_selected_dataset,model.decoder_cfg.should_share,model.weight_init.gain,optimizer.momentum,model.pretrained_cfg.should_use,model.weight_init.bias,model.encoder_cfg.should_share,mean_average_accuracy_for_unselected_paths,stderr_average_accuracy_for_unselected_paths,steps,seeds
0,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,1000,0.0001,40,False,True,8,10_plus_mod_permute,10,False,0.1000,0.9,False,0,False,0.798361,0.019675,1000.0,2
1,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,1000,0.0001,40,False,True,8,10_plus_mod_permute,10,False,1.0000,0.9,False,0,False,0.779538,0.001894,1000.0,2
2,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,1000,0.0001,40,False,True,8,10_plus_mod_permute,10,False,10.0000,0.9,False,0,False,0.158561,0.000167,1000.0,2
3,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,1000,0.0001,40,False,True,8,10_plus_mod_permute,10,False,0.0100,0.9,False,0,False,0.113500,0.000000,1000.0,2
4,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,1000,0.0001,40,False,True,8,10_plus_mod_permute,10,False,0.0001,0.9,False,0,False,0.113500,0.000000,1000.0,2
5,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,1000,0.0001,40,False,True,8,10_plus_mod_permute,10,False,0.0010,0.9,False,0,False,0.113500,0.000000,1000.0,2
6,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,1000,0.0001,40,False,True,8,35_plus_mod_permute,10,False,0.1000,0.9,False,0,False,0.911173,0.000000,1000.0,1
7,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,1000,0.0001,40,False,True,8,35_plus_mod_permute,10,False,1.0000,0.9,False,0,False,0.924536,0.000000,1000.0,1
8,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,1000,0.0001,40,False,True,8,35_plus_mod_permute,10,False,10.0000,0.9,False,0,False,0.898623,0.000000,1000.0,1
9,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,1000,0.0001,40,False,True,8,35_plus_mod_permute,10,False,0.0100,0.9,False,0,False,0.113500,0.000000,1000.0,1


In [24]:
keys_to_merge_on = [
    "model.hidden_layer_cfg.dim",
    "experiment.task.num_classes_in_selected_dataset",
    "experiment.num_epochs",
    "dataloader.name",
    "model.decoder_cfg.should_share",
    "model.weight_init.bias",
    "experiment.task.mode",
    "model.num_layers",
    "optimizer._target_",
    "setup.script_id",
    "model.weight_init.gain",
    "model.weight_init.should_do",
    "model.encoder_cfg.should_share",
    "experiment.task.num_input_transformations",
    "optimizer.lr",
    "optimizer.momentum",
    "model.should_use_non_linearity",
    "model.gate_cfg.mode",
    "dataloader.train_config.dataloader.batch_size",
    "model.pretrained_cfg.should_use",
    "model.hidden_layer_cfg.should_share",
    "steps",
    "seeds",
#     "model.hidden_layer_cfg.num_layers",
#     "model.hidden_layer_cfg.should_use_non_linearity",
#     "experiment.name",
    
    
#     "model.encoder_cfg.should_share",
# #     "experiment.name",
#     "model.weight_init.should_do",
#     "model.weight_init.gain",
#     "model.weight_init.bias",
#     "optimizer._target_",
#     "optimizer.lr",
#     "model.hidden_layer_cfg.dim",
#     "model.should_use_non_linearity",
#     "model.hidden_layer_cfg.should_share",
#     "model.num_layers",
#     
#     "experiment.task.num_classes_in_selected_dataset",
#     "model.gate_cfg.mode",
#     "experiment.num_epochs",
#     "dataloader.train_config.dataloader.batch_size",
#     "optimizer.momentum"
]
merged_df = df_dict[metrics[0]]
for current_metric in metrics[1:]:
    merged_df = merged_df.merge(
        df_dict[current_metric], on=keys_to_merge_on, suffixes=[None, None]
    )

merged_df = merged_df.drop(
    [
        "seeds",
        "steps",
#         "experiment.name",
#         "experiment.task.num_classes_in_original_dataset",
#         "model.gate_cfg.mode",
        "experiment.num_epochs",
        "mean_average_loss_for_unselected_paths",
        "mean_average_loss_for_selected_paths"
    ]
    + [f"stderr_{metric_name}" for metric_name in metrics],
    axis=1,
)

merged_df.style.set_sticky(axis="columns")

merged_df

# import seaborn as sns

# cm = sns.light_palette("red", as_cmap=True)

# merged_df.style.background_gradient(
#     axis=1, subset=[f"mean_return_mean_{key}" for key in keys], vmax=1.4, vmin=0.6
# )

,setup.script_id,model.hidden_layer_cfg.dim,experiment.task.mode,model.num_layers,model.weight_init.should_do,dataloader.name,optimizer._target_,optimizer.lr,experiment.task.num_input_transformations,model.should_use_non_linearity,model.hidden_layer_cfg.should_share,dataloader.train_config.dataloader.batch_size,model.gate_cfg.mode,experiment.task.num_classes_in_selected_dataset,model.decoder_cfg.should_share,model.weight_init.gain,optimizer.momentum,model.pretrained_cfg.should_use,model.weight_init.bias,model.encoder_cfg.should_share,mean_average_accuracy_for_selected_paths,mean_average_accuracy_for_unselected_paths
0,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,0.0001,40,False,True,8,10_plus_mod_permute,10,False,0.1000,0.9,False,0,False,0.916342,0.798361
1,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,0.0001,40,False,True,8,10_plus_mod_permute,10,False,1.0000,0.9,False,0,False,0.924467,0.779538
2,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,0.0001,40,False,True,8,10_plus_mod_permute,10,False,10.0000,0.9,False,0,False,0.911426,0.158561
3,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,0.0001,40,False,True,8,10_plus_mod_permute,10,False,0.0100,0.9,False,0,False,0.113500,0.113500
4,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,0.0001,40,False,True,8,10_plus_mod_permute,10,False,0.0001,0.9,False,0,False,0.113500,0.113500
5,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,0.0001,40,False,True,8,10_plus_mod_permute,10,False,0.0010,0.9,False,0,False,0.113500,0.113500
6,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,0.0001,40,False,True,8,35_plus_mod_permute,10,False,0.1000,0.9,False,0,False,0.911175,0.911173
7,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,0.0001,40,False,True,8,35_plus_mod_permute,10,False,1.0000,0.9,False,0,False,0.924827,0.924536
8,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,0.0001,40,False,True,8,35_plus_mod_permute,10,False,10.0000,0.9,False,0,False,0.907658,0.898623
9,25,1024,rotate_input_permute_target,1,True,mnist,torch.optim.SGD,0.0001,40,False,True,8,35_plus_mod_permute,10,False,0.0100,0.9,False,0,False,0.113500,0.113500


In [25]:
df_to_plot = merged_df

# keys_to_drop = ["model.should_share_hidden_layer", "model.name", "model.should_share_decoder", 
#                 "stderr_accuracy_greater_than_four_encoder_odd_even_decoder", 
#                 "steps", "seeds", 
#                 "model.num_layers", "model.weight_init.should_do", 
#                 "model.should_share_encoder", "model.hidden_size"
#                ]
# df_to_plot = df_to_plot.drop(keys_to_drop, axis=1)

# hip.Experiment.from_dataframe(df_to_plot).display()



In [26]:
# experiment.task.num_classes_in_selected_dataset
# 	{10}
# model.hidden_layer_cfg.should_share
# 	{True}
# optimizer._target_
# 	{'torch.optim.SGD'}
# model.decoder_cfg.should_share
# 	{False}
# dataloader.train_config.dataloader.batch_size
# 	{8}
# dataloader.name
# 	{'mnist'}
# model.num_layers
# 	{1}
# model.should_use_non_linearity
# 	{False}
# optimizer.lr
# 	{0.0001}
# model.encoder_cfg.should_share
# 	{False}
# optimizer.momentum
# 	{0.9}
# model.hidden_layer_cfg.dim
# 	{1024, 128}
# experiment.num_epochs
# 	{1000}
# model.pretrained_cfg.should_use
# 	{False}
# setup.script_id
# 	{25}
# experiment.task.num_input_transformations
# 	{40}
# model.weight_init.bias
# 	{0}
# model.weight_init.should_do
# 	{True}
# model.weight_init.gain
# 	{0.1, 1.0, 10.0, 0.01, 0.0001, 0.001}
# model.gate_cfg.mode
# 	{'30_plus_mod_permute', '10_plus_mod_permute', '20_plus_mod_permute', '35_plus_mod_permute', '5_plus_mod_permute', '25_plus_mod_permute', '15_plus_mod_permute'}

# 	{'rotate_input_permute_target', 'permute_input_permute_target'}

IndentationError: unexpected indent (1133417695.py, line 2)

In [27]:
%matplotlib inline
pd.set_option("plotting.backend", "pandas_bokeh")
pandas_bokeh.output_notebook()

%matplotlib inline
# pd.set_option("plotting.backend", "pandas_bokeh")
# sns.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
# pandas_bokeh.output_notebook()
sns.color_palette("deep")

plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

Loading BokehJS ...

In [28]:
%debug

svd_values = {
    record.setup.id: svd_utils.compute_svd_values(record) for record in record_list
}


> /private/home/sodhani/.conda/envs/abstraction_by_gating/lib/python3.9/site-packages/IPython/core/compilerop.py(101)ast_parse()
     99         Arguments are exactly the same as ast.parse (in the standard library),
    100         and are passed to the built-in compile function."""
--> 101         return compile(source, filename, symbol, self.flags | PyCF_ONLY_AST, 1)
    102 
    103     def reset_compiler_flags(self):



ipdb>  c


InstantiationException: Error locating target 'src.experiment.k_path_model.Experiment', see chained exception above.
full_key: experiment.builder

In [29]:
res = {}
for val in exp_seq_dict.values():
    val = val[0]
    setup_id = val.config["setup"]["id"]
    res[setup_id] = {
        "average_accuracy_for_unselected_paths": val.metrics["test_epoch"]["average_accuracy_for_unselected_paths"].to_numpy()[-1],
        "average_accuracy_for_selected_paths": val.metrics["test_epoch"]["average_accuracy_for_selected_paths"].to_numpy()[-1],
        "average_loss_for_unselected_paths": val.metrics["test_epoch"]["average_loss_for_unselected_paths"].to_numpy()[-1],
        "average_loss_for_selected_paths": val.metrics["test_epoch"]["average_loss_for_selected_paths"].to_numpy()[-1],
        "svd_encoder": torch.cat([x[0] for x in svd_values[setup_id].encoder], 0).max().item(),
        "svd_decoder": torch.cat(svd_values[setup_id].decoder, 0).max().item(),
        "svd_hidden": svd_values[setup_id].hidden[0].max().item(),
        "K": int(val.config["model"]["gate_cfg"]["mode"].split("_")[0]),
        "M": 40,
        "gain": val.config["model"]["weight_init"]["gain"],
        "dim": val.config["model"]["hidden_layer_cfg"]["dim"],
        "mode": val.config["experiment"]["task"]["mode"],
    }


NameError: name 'torch' is not defined

In [30]:
df = pd.DataFrame.from_dict(res, orient="index")
df = df.reset_index().drop(["index"], axis=1)
df["scaled_km"] = np.power((df["K"] * df["M"]).to_numpy(), 0.25)
df["sqrt_m"] = np.power(df["M"].to_numpy(), 0.5)

KeyError: 'K'

In [31]:
df = df[df["average_accuracy_for_unselected_paths"] > 0.92]
df = df[df["dim"] == 128]
# df = df[df["mode"][]

df["svd_encoder_intial"] = 1.82 * df["gain"]
df["svd_hidden_intial"] = 1.96 * df["gain"]
df["svd_decoder_intial"] = 1.65 * df["gain"]
# df["ratio_hidden_encoder"] = (df["svd_hidden"] - 1.96 * df["gain"]) / (df["svd_encoder"] - 1.82 * df["gain"])
# df["ratio_hidden_decoder"] = (df["svd_hidden"] - 1.96 * df["gain"]) / (df["svd_decoder"] - 1.65 * df["gain"])

df["ratio_hidden_encoder"] = (df["svd_hidden"]) / (df["svd_encoder"])
df["ratio_hidden_decoder"] = (df["svd_hidden"]) / (df["svd_decoder"])

KeyError: 'average_accuracy_for_unselected_paths'

In [32]:
# df[df["gain"] == 0.1]
df

""


In [33]:
df.plot_bokeh("")

Exception: Please provide for the <x> parameter either a column name of the DataFrame/Series or an array of the same length.

In [34]:
import scipy
scipy.stats.pearsonr(df["M"] * df["K"], df["ratio_hidden_encoder"])

KeyError: 'M'

In [35]:
import torch

In [36]:
def get_weight_init_fn(gain: float, bias: float = 0.01):
    def init_weights(m):
        if isinstance(m, (torch.nn.Linear)):
            torch.nn.init.xavier_uniform_(
                m.weight,
                gain=gain,
            )
            m.bias.data.fill_(bias)
        elif isinstance(
            m,
            (
                torch.nn.Flatten,
                torch.nn.ReLU,
                torch.nn.LeakyReLU,
                torch.nn.Sequential,
                torch.nn.ModuleList,
            ),
        ):
            pass
        else:
            raise NotImplementedError(f"module = {m} is not supported.")

    return init_weights

In [37]:
model = torch.nn.Linear(784, 128)

for gain in [100, 10, 1, 0, 0.1, 0.01, 0.001]:
    svd = []
    for seed in range(100):
        init_weights = get_weight_init_fn(gain, 0)
        model.apply(init_weights)
        svd.append(torch.svd(model.weight)[1].max().item())
    print(f"gain = {gain}, svd = {torch.tensor(svd).mean()}")

gain = 100, svd = 182.24453735351562
gain = 10, svd = 18.22764015197754
gain = 1, svd = 1.8225401639938354
gain = 0, svd = 0.0
gain = 0.1, svd = 0.18234437704086304
gain = 0.01, svd = 0.01821262761950493
gain = 0.001, svd = 0.0018239544006064534


In [38]:
model = torch.nn.Linear(128, 128)

for gain in [100, 10, 1, 0, 0.1, 0.01, 0.001]:
    svd = []
    for seed in range(100):
        init_weights = get_weight_init_fn(gain, 0)
        model.apply(init_weights)
        svd.append(torch.svd(model.weight)[1].max().item())
    print(f"gain = {gain}, svd = {torch.tensor(svd).mean()}")

gain = 100, svd = 195.5329132080078
gain = 10, svd = 19.522777557373047
gain = 1, svd = 1.953436017036438
gain = 0, svd = 0.0
gain = 0.1, svd = 0.1956554353237152
gain = 0.01, svd = 0.019555391743779182
gain = 0.001, svd = 0.0019542935770004988


In [39]:
model = torch.nn.Linear(128, 10)

for gain in [100, 10, 1, 0, 0.1, 0.01, 0.001]:
    svd = []
    for seed in range(100):
        init_weights = get_weight_init_fn(gain, 0)
        model.apply(init_weights)
        svd.append(torch.svd(model.weight)[1].max().item())
    print(f"gain = {gain}, svd = {torch.tensor(svd).mean()}")

gain = 100, svd = 165.30752563476562
gain = 10, svd = 16.559782028198242
gain = 1, svd = 1.6610920429229736
gain = 0, svd = 0.0
gain = 0.1, svd = 0.1655239313840866
gain = 0.01, svd = 0.01651495322585106
gain = 0.001, svd = 0.0016530645079910755
